In [ ]:
!pip install httpx
!pip install bitsandbytes
!pip install accelerate
!pip install typing

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from pathlib import Path
import torch
import torch.nn as nn
import torchvision.models as models
from transformers import AutoProcessor, Blip2ForConditionalGeneration, BitsAndBytesConfig, AdamW, get_scheduler
from typing import Tuple, Union
import httpx
from bs4 import BeautifulSoup
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

In [3]:
class BLIPPromptTuner:

  # @ classmethod
  def __init__(self): #alt_init(self,**kwargs):

    self.num_train_epochs = 1
    self.weight_decay = 0.01
    self.learning_rate = 0.01
    self.lr_scheduler_type = "linear"
    self.num_warmup_steps = 0
    self.max_train_steps = 1 #num_train_epochs
    self.n_embd = 768
    self.device = "cuda" if torch.cuda.is_available() else "cpu"
    # Prompt-tuning
    # number of prompt tokens
    self.n_prompt_tokens = 20
    # If True, soft prompt will be initialized from vocab
    # Otherwise, you can set `random_range` to initialize by randomization.
    self.init_from_vocab = True
    # random_range = 0.5

    from transformers import BitsAndBytesConfig
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    self.processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
    self.model = Blip2ForConditionalGeneration.from_pretrained("ybelkada/blip2-opt-2.7b-fp16-sharded",
                                                                device_map={"":0},
                                                                quantization_config=bnb_config)#load_in_8bit=True)

    # freeze model
    for param in self.model.parameters():
        param.requires_grad = False

  def init_model(self):
    print("Initializing soft prompt...")
    self.init_soft_prompt_embeds() #model.init_soft_prompt_embeds()
    return self.model

  def set_soft_prompt_embeds(
      self,
      soft_prompt_path: str,
  ) -> None:
      """
      Args:
          soft_prompt_path: torch soft prompt file path

      """
      self.soft_prompt = torch.load(
          soft_prompt_path, map_location=torch.device("cuda")
      )
      self.n_tokens = self.soft_prompt.num_embeddings
      print(f"Set soft prompt! (n_tokens: {self.n_tokens})")

  def init_soft_prompt_embeds(self,
                              n_tokens: int = 20,
                              initialize_from_vocab: bool = True,
                              random_range: float = 0.5,
                              ) -> None:
    self.n_tokens = n_tokens
    if initialize_from_vocab:
        init_prompt_value = self.model.language_model.get_input_embeddings().weight[:n_tokens].clone().detach() #self.transformer.wte.weight[:n_tokens].clone().detach()
    else:
        init_prompt_value = torch.FloatTensor(2, 10).uniform_(
            -random_range, random_range
        )
    self.soft_prompt = nn.Embedding(n_tokens, self.n_embd)#config.n_embd)
    # Initialize weight
    self.soft_prompt.weight = nn.parameter.Parameter(init_prompt_value)
    # add the layer to the pretrained model
    self.model = nn.Sequential(self.model,
                               self.soft_prompt)

  def _cat_learned_embedding_to_input(self, input_ids) -> torch.Tensor:
      print()
      inputs_embeds = self.model[0].language_model.get_input_embeddings()(input_ids) #self.transformer.wte(input_ids) #model.language_model.get_input_embeddings().weight
      if len(list(inputs_embeds.shape)) == 2:
          inputs_embeds = inputs_embeds.unsqueeze(0)
      # [batch_size, n_tokens, n_embd]
      learned_embeds = self.soft_prompt.weight.repeat(inputs_embeds.size(0), 1, 1)
      inputs_embeds = torch.cat([learned_embeds, inputs_embeds], dim=1)

      return inputs_embeds

  def _extend_labels(self, labels, ignore_index=-100) -> torch.Tensor:
      if len(list(labels.shape)) == 1:
          labels = labels.unsqueeze(0)

      n_batches = labels.shape[0]
      return torch.cat(
          [
              torch.full((n_batches, self.n_tokens), ignore_index).to(self.device),
              labels.to(self.device),
          ],
          dim=1,
      )

  def _extend_attention_mask(self, attention_mask):

      if len(list(attention_mask.shape)) == 1:
          attention_mask = attention_mask.unsqueeze(0)

      n_batches = attention_mask.shape[0]
      return torch.cat(
          [torch.full((n_batches, self.n_tokens), 1).to(self.device), attention_mask.to(self.device)],
          dim=1,
      )

  def forward(
      self,
      pixel_values: torch.FloatTensor,
      input_ids: torch.FloatTensor,
      attention_mask = None,
      decoder_input_ids = None,
      decoder_attention_mask = None,
      output_attentions = None,
      output_hidden_states = None,
      labels = None,
      return_dict = None,
  ):

      from transformers.models.blip_2.modeling_blip_2 import Blip2ForConditionalGenerationModelOutput


      if labels is not None:
          labels = self._extend_labels(labels).to(self.device)

      if attention_mask is not None:
          attention_mask = self._extend_attention_mask(attention_mask).to(self.device)

      # start of the hugging face forward function
      return_dict = return_dict if return_dict is not None else self.model[0].config.use_return_dict

      # step 1: forward the images through the vision encoder,
      # to get image embeddings of shape (batch_size, seq_len, hidden_size)
      vision_outputs = self.model[0].vision_model(
          pixel_values=pixel_values,
          output_attentions=output_attentions,
          output_hidden_states=output_hidden_states,
          return_dict=return_dict,
      )
      image_embeds = vision_outputs[0]

      # step 2: forward the query tokens through the QFormer, using the image embeddings for cross-attention
      image_attention_mask = torch.ones(image_embeds.size()[:-1], dtype=torch.long, device=image_embeds.device)

      query_tokens = self.model[0].query_tokens.expand(image_embeds.shape[0], -1, -1)
      query_outputs = self.model[0].qformer(
          query_embeds=query_tokens,
          encoder_hidden_states=image_embeds,
          encoder_attention_mask=image_attention_mask,
          output_attentions=output_attentions,
          output_hidden_states=output_hidden_states,
          return_dict=return_dict,
      )
      query_output = query_outputs[0]

      # step 3: use the language model, conditioned on the query outputs and the prompt
      language_model_inputs = self.model[0].language_projection(query_output)
      language_model_attention_mask = torch.ones(
          language_model_inputs.size()[:-1], dtype=torch.long, device=language_model_inputs.device
      )
      # HOW ORGINAL EMBEDDINGS WERE CALCULATED
      # inputs_embeds = self.model[0].language_model.get_input_embeddings()(input_ids)
      # inputs_embeds = torch.cat([language_model_inputs, inputs_embeds.to(language_model_inputs.device)], dim=1)

      if input_ids is not None: # this represents the textual input embeddings with the concat. soft prompts
          inputs_embeds = self._cat_learned_embedding_to_input(input_ids).to(
              self.device
          )

      if attention_mask is None: # we dont need to change this?
          attention_mask = torch.ones_like(input_ids)
      expected_device = language_model_attention_mask.device
      attention_mask = torch.cat([language_model_attention_mask, attention_mask.to(expected_device)], dim=1)

      if self.model[0].config.use_decoder_only_language_model:
          outputs = self.language_model(
              inputs_embeds=inputs_embeds,
              attention_mask=attention_mask,
              output_attentions=output_attentions,
              output_hidden_states=output_hidden_states,
              return_dict=return_dict,
          )
          logits = outputs.logits if return_dict else outputs[0]
          loss = None
          # we compute the loss here since we need to take into account the sequence length of the query embeds
          if labels is not None:
              labels = labels.to(logits.device)
              logits = logits[:, -labels.size(1) :, :]
              # Shift so that tokens < n predict n
              shift_logits = logits[..., :-1, :].contiguous()
              shift_labels = labels[..., 1:].contiguous().to(logits.device)

              # Flatten the tokens
              loss_fct = nn.CrossEntropyLoss(reduction="mean")

              loss = loss_fct(shift_logits.view(-1, self.model[0].config.text_config.vocab_size), shift_labels.view(-1))
      else:
          outputs = self.model[0].language_model(
              inputs_embeds=inputs_embeds,
              attention_mask=attention_mask,
              decoder_input_ids=decoder_input_ids,
              decoder_attention_mask=decoder_attention_mask,
              output_attentions=output_attentions,
              output_hidden_states=output_hidden_states,
              return_dict=return_dict,
              labels=labels,
          )
          loss = outputs.loss if return_dict else outputs[0]
          logits = outputs.logits if return_dict else outputs[1]

      if not return_dict:
          output = (logits, vision_outputs, query_outputs, outputs)
          return ((loss,) + output) if loss is not None else output

      return Blip2ForConditionalGenerationModelOutput(
          loss=loss,
          logits=logits,
          vision_outputs=vision_outputs,
          qformer_outputs=query_outputs,
          language_model_outputs=outputs,
      )

In [4]:
class BLIPPromptTunerFinal(BLIPPromptTuner, Blip2ForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)

In [ ]:
# download the image files themselves (the original ZIP FILE IS 25G so we'll scrape a small subset lol)


df = pd.read_csv('Contrastive.csv')

base_url = "https://uploads8.wikiart.org/images/"
stopper = 5
valid_images_count = 0
invalid_images=[]

for i, item in enumerate(df.iterrows()):
    item_name_spl = item[1]['painting'].split('_')
    painter = item_name_spl[0]
    painting = item_name_spl[1]
    scrape_url = base_url + painter + '/' + painting + '.jpg!Large.jpg'

    try:
        response = httpx.get(scrape_url)
        if response.status_code == 200 and response.headers['Content-Type'].startswith('image'):
            with open('imgs/' + item[1]['painting'] + '.jpeg', 'wb') as outf:
                outf.write(response.content)
            valid_images_count += 1
            print(f"Downloaded image {valid_images_count}: {item[1]['painting']}")
        else:
            print(f"Skipped invalid image URL: {scrape_url}")
            invalid_images.append(i)
    except httpx.RequestError as e:
        print(f"Error downloading image: {scrape_url}")
        print(f"Error message: {str(e)}")

    if valid_images_count == stopper:
        break

with open('imgs/invalid_images.txt', 'w') as outfile:
    for number in invalid_images:
        outfile.write(str(number) + '\n')

NameError: name 'pd' is not defined

In [6]:
instance = BLIPPromptTuner()
instance.init_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: b326d980-1f7f-49a4-9520-2b93128a464b)')

In [ ]:
# # testing stuff
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
test_model = Blip2ForConditionalGeneration.from_pretrained("ybelkada/blip2-opt-2.7b-fp16-sharded",
                                                       device_map="auto",
                                                       quantization_config=bnb_config)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [15]:
# define a dataset class

class ArtemisDataset(Dataset):
    def __init__(self, processor, labels_fname="/content/drive/MyDrive/Contrastive.csv", img_dirname="/content/drive/MyDrive/imgs"):
        self.img_labels = pd.read_csv(labels_fname).iloc[:25]
        self.img_dirname = img_dirname
        self.processor = processor
        self.vocab = list(self.processor.tokenizer.get_vocab().keys())

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        painting_name = self.img_labels.iloc[idx]['painting'] + '.jpeg'
        print(painting_name)
        img_path = os.path.join(self.img_dirname, painting_name)
        # need to make sure that we are reading an image that's valid
        image = read_image(img_path)
        label = self.img_labels.iloc[idx]['utterance']
        emotion = self.img_labels.iloc[idx]['emotion']

        label= f"This is a caption that invokes {emotion}: {label}"

        encoding = self.processor(images=image, padding="max_length", return_tensors="pt")
        # remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}
        encoding["text"] = label
        return encoding

def collate_fn(batch):
  # pad the input_ids and attention_mask
  processed_batch = {}
  for key in batch[0].keys():
      if key != "text":
          processed_batch[key] = torch.stack([example[key] for example in batch])
      else:
          text_inputs = instance.processor.tokenizer(
              [example["text"] for example in batch], padding=True, return_tensors="pt"
          )
          processed_batch["input_ids"] = text_inputs["input_ids"]
          processed_batch["attention_mask"] = text_inputs["attention_mask"]
  return processed_batch



In [16]:
# test_emb = nn.Embedding(16,16)
#test_wte = nn.parameter.Parameter(0)
#   self.soft_prompt = nn.Embedding(n_tokens, self.n_embd)#config.n_embd)
#   # Initialize weight
#   self.soft_prompt.weight = nn.parameter.Parameter(init_prompt_value)
#   # add the layer to the pretrained model
#   self.model = nn.Sequential(self.model,
#                               self.soft_prompt.weight)
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in instance.model.named_parameters() if n == "1.weight" ],
        "weight_decay": instance.weight_decay,
    }
]
optimizer = torch.optim.Adam(optimizer_grouped_parameters, lr=1e-2)

train_dataset = ArtemisDataset(processor=instance.processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=5, collate_fn=collate_fn)
instance.model.train()
for epoch in range(3):
  print("Epoch: ", epoch)
  for idx, batch in enumerate(train_dataloader):
    loss = instance.forward(**batch, labels=batch['input_ids'])
    print("Loss:", loss.item())
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

Epoch:  0
ivan-aivazovsky_seascape-in-crimea-1866.jpeg
ivan-aivazovsky_windmill-on-the-sea-coast-1851.jpeg
cornelis-springer_janskerk-gouda-1863.jpeg
alfred-sisley_bank-in-the-autumn-1879.jpeg
johan-hendrik-weissenbruch_ships-on-canal.jpeg


TypeError: Blip2ForConditionalGeneration.forward() got an unexpected keyword argument 'inputs_embeds'